# Classes

## Create new instance of class
[object.__new__(cls[, ...])](https://docs.python.org/3/reference/datamodel.html#object.__new__)

In [5]:
import random


class Singleton:
    
    def __new__(cls, *args, **kwargs):
        print(f"Run class constructor for class {cls.__class__}, args: {args}, kwargs: {kwargs}")
        
        if not hasattr(cls, '_instance'):
            print("Define new class instance")
            cls._instance = super().__new__(cls, *args, **kwargs)
            
        if not isinstance(cls._instance, cls):
            raise TypeError(f"Instance of class {cls._instance} but expected {cls.__class__}")
            
        return cls._instance

    def __init__(self, *args, **kwargs):
        print(f"Initialise class {self.__class__}, instance {self._instance}, args: {args}, kwargs: {kwargs}")
        self._seed = random.randint(0, 100)
        
    @property
    def instance(self):
        return self._instance
    
    @property
    def seed(self):
        return self._seed


In [2]:
s1 = Singleton()

id(s1), s1.instance, s1.seed

Run class constructor for class <class 'type'>, args: (), kwargs: {}
Define new class instance
Initialise class <class '__main__.Singleton'>, instance <__main__.Singleton object at 0x7fb288ac9430>, args: (), kwargs: {}


(140404773917744, <__main__.Singleton at 0x7fb288ac9430>, 20)

In [3]:
s2 = Singleton()

id(s2), s2.instance, s2.seed

Run class constructor for class <class 'type'>, args: (), kwargs: {}
Initialise class <class '__main__.Singleton'>, instance <__main__.Singleton object at 0x7fb288ac9430>, args: (), kwargs: {}


(140404773917744, <__main__.Singleton at 0x7fb288ac9430>, 55)

In [4]:
id(s1), s1.instance, s1.seed

(140404773917744, <__main__.Singleton at 0x7fb288ac9430>, 55)

### object.call

In [6]:
class Calculation:
    def __init__(self, op="+"):
        print(f"Initialise class {self.__class__}")
        self.op = op
        
    def __call__(self, a, b):
        print(f"Call class {self.__class__}")
        if self.op == "+":
            return a + b
        elif self.op == "-":
            return a - b
        
        raise TypeError

In [7]:
calc = Calculation()

Initialise class <class '__main__.Calculation'>


In [8]:
calc.__call__(1, 2)

Call class <class '__main__.Calculation'>


3

In [9]:
calc(1, 2)

Call class <class '__main__.Calculation'>


3

In [10]:
Calculation("-")(1, 2)

Initialise class <class '__main__.Calculation'>
Call class <class '__main__.Calculation'>


-1

In [11]:
Calculation("-").__call__(1, 2)

Initialise class <class '__main__.Calculation'>
Call class <class '__main__.Calculation'>


-1

## Abstract classes

In [12]:
class Employee:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def position(self):
        raise NotImplementedError


In [13]:
emp = Employee("Eve", 18)

In [14]:
emp.position()

NotImplementedError: 

In [15]:
class Manager(Employee):
    def position(self):
        return "Manager"
    
class Developer(Employee):
    def position(self):
        return "Developer"

In [16]:
dev = Developer("Alice", 22)
dev.name, dev.age, dev.position()


('Alice', 22, 'Developer')

In [17]:
manager = Manager("Bob", 25)
manager.name, manager.age, manager.position()


('Bob', 25, 'Manager')

### abc — Abstract Base Classes

https://docs.python.org/3/library/abc.html#module-abc

In [18]:
from abc import ABCMeta, abstractmethod


class Employee(metaclass=ABCMeta):
    def __init__(self, name, age):
        self.name = name
        self.age = age
            
    @abstractmethod
    def position(self):
        pass

In [19]:
emp = Employee("Eve", 18)

TypeError: Can't instantiate abstract class Employee with abstract methods position

In [20]:
class Manager(Employee):
    def position(self):
        return "Manager"
    
class Developer(Employee):
    def position(self):
        return "Developer"

In [21]:
dev = Developer("Alice", 22)
dev.name, dev.age, dev.position()


('Alice', 22, 'Developer')

In [22]:
manager = Manager("Bob", 25)
manager.name, manager.age, manager.position()


('Bob', 25, 'Manager')

### Abstract Base Classes for Containers

[collections.abc](https://docs.python.org/3/library/collections.abc.html)

In [5]:
jd_raw = '{"a": 1, "b": "XXX", "c": [1, 2, 3]}'

In [16]:
class ExtendedMapping(Mapping):
    def __init__(self, m):
        self.m = m
        if not isinstance(m, Mapping):
            raise TypeError

    def __len__(self) -> int:
        return len(self.m)

    def __iter__(self) -> Mapping:
        yield from self.m

    def __getitem__(self, key: str) -> str:
        print(type(key))
        return self.m[key]

In [2]:
from collections.abc import Mapping
import json


class Json2Dict(Mapping):
    def __init__(self, json_string):
        j = json.loads(json_string)
        
        if not isinstance(j, Mapping):
            raise ValueError(f"Expected json-object but encoded to {type(j)}")

        self._json_object = j

    def __getitem__(self, k):
        return self._json_object[k]

    def __len__(self):
        return len(self._json_object)

    def __iter__(self):
        yield from self._json_object

In [6]:
jd = Json2Dict(jd_raw)

jd

In [27]:
jd["a"]

1

In [7]:
d1 = {'a': {'b': {'c': 1, 'd': 2}, 'e': 3}, 'f': 4}

In [17]:
 d=ExtendedMapping(d1)

In [18]:
d['a']

<class 'str'>


{'b': {'c': 1, 'd': 2}, 'e': 3}

In [21]:
d.a


SyntaxError: invalid syntax (<ipython-input-21-6318e03728ec>, line 1)

In [28]:
jd["b"]

'XXX'

In [29]:
jd["c"]

[1, 2, 3]

In [30]:
jd["d"]

KeyError: 'd'

In [31]:
len(jd)

3

In [32]:
for x in jd:
    print(x)

a
b
c


In [33]:
for k, v in jd.items():
    print(k, v)

a 1
b XXX
c [1, 2, 3]


In [34]:
"d" in jd

False

In [35]:
"a" in jd

True

In [36]:
dict(jd)

{'a': 1, 'b': 'XXX', 'c': [1, 2, 3]}

In [37]:
{**jd}

{'a': 1, 'b': 'XXX', 'c': [1, 2, 3]}